In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install shutil

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,095 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,519 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://

In [2]:
from google.colab import drive
drive.mount('/content/drive') # mounta google drive

# /content/common_voice
!tar -xzvf "drive/MyDrive/common_voice.tar.gz" -C .

Mounted at /content/drive
common_voice/
common_voice/train/
common_voice/train/state.json
common_voice/train/dataset.arrow
common_voice/train/dataset_info.json
common_voice/dataset_dict.json
common_voice/test/
common_voice/test/state.json
common_voice/test/dataset.arrow
common_voice/test/dataset_info.json


In [3]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict.load_from_disk("common_voice")

In [4]:
print(common_voice["train"])
print(common_voice["test"])

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2500
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 800
})


In [5]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [6]:
# create processor, extractor, tokenizer if preparation step skipped
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

In [7]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [8]:
import evaluate

metric = evaluate.load("wer")

In [9]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

In [11]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [12]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-sv-SE-auhr-v2",  # change to a repo name of your choice
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,  
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

# write token here
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [13]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/repository.py:725: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  warnings.warn(
Cloning https://huggingface.co/Akseluhr/whisper-small-sv-SE-auhr-v2 into local empty directory.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [14]:
processor.save_pretrained(training_args.output_dir)

Feature extractor saved in ./whisper-small-sv-SE-auhr-v2/preprocessor_config.json
tokenizer config file saved in ./whisper-small-sv-SE-auhr-v2/tokenizer_config.json
Special tokens file saved in ./whisper-small-sv-SE-auhr-v2/special_tokens_map.json
added tokens file saved in ./whisper-small-sv-SE-auhr-v2/added_tokens.json


Training will take approximately 5-10 hours depending on your GPU or the one allocated to this Google Colab. If using this Google Colab directly to fine-tune a Whisper model, you should make sure that training isn't interrupted due to inactivity. A simple workaround to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab -> insert code).

In [ ]:
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton, 60000);

In [15]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2500
  Num Epochs = 52
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 16
  Total optimization steps = 4000
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = Tr

Step,Training Loss,Validation Loss,Wer
500,0.020100,0.431915,26.831262
1000,0.001400,0.482853,27.174624


Streaming output truncated to the last 5000 lines.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...

KeyboardInterrupt: ignored

In [16]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "sv-SE",
    "model_name": "Whisper Test SE v2 - Aksel Uhr",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [17]:
trainer.push_to_hub(**kwargs) # pushes to my hub...

Saving model checkpoint to ./whisper-small-sv-SE-auhr-v2
Configuration saved in ./whisper-small-sv-SE-auhr-v2/config.json
Model weights saved in ./whisper-small-sv-SE-auhr-v2/pytorch_model.bin
Feature extractor saved in ./whisper-small-sv-SE-auhr-v2/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/922M [00:00<?, ?B/s]

Upload file runs/Dec08_08-23-48_1d97925cd8f2/events.out.tfevents.1670487871.1d97925cd8f2.76.0:  29%|##9       …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Akseluhr/whisper-small-sv-SE-auhr-v2
   b416f89..df8d143  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Akseluhr/whisper-small-sv-SE-auhr-v2
   b416f89..df8d143  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Automatic Speech Recognition', 'type': 'automatic-speech-recognition'}, 'dataset': {'name': 'Common Voice 11.0', 'type': 'mozilla-foundation/common_voice_11_0', 'config': None, 'split': 'None', 'args': 'config: hi, split: test'}}
remote: ----------------------------------------------------------        
remote: Sorry, your push was rejected during YAML metadata verification:        
remote: - Error: "language[0]" must only contain lowercase characters        
remote: - Error: "language[0]" with value "sv-SE" is not valid. It must be an ISO 639-1, 639-2 or 639

'https://huggingface.co/Akseluhr/whisper-small-sv-SE-auhr-v2/commit/df8d14388fd12d57f442b472669a56b136e56d5b'

In [21]:
# /content/whisper-small-sv-SE-auhr-v2
import shutil
#from google.colab import drive
#drive.mount('/content/drive')
#shutil.copy("/content/whisper-small-sv-SE-auhr-v2", "/content/drive/MyDrive/whisper-small-sv-SE-auhr-v2") 

!mv "/content/whisper-small-sv-SE-auhr-v2" "/content/drive/MyDrive/whisper-small-sv-SE-auhr-v2"
